The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [4]:
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [23]:
dataset = "a9a"   # news20.binary  rcv1_train.multiclass rcv1_train.binary mnist.scale  a9a  gisette_scale 
prob = defineProb(dataset);
showDetails(dataset)

../data/a9a.jld
ErrorException("File ../data/a9a-xsol.jld cannot be found")


Dict{Symbol,Any} with 3 entries:
  :sparsity => 0.112757
  :dims     => 123
  :n        => 32561

In [24]:
norm(prob.A*prob.xsol - prob.b)

1.967333493545081e-11

In [33]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [34]:
options.AUX = [0 2]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = floor((prob.n)^(1/2)); #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 5; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 2; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-11-2
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 1.05  |  1.22  | 0.0041 
 50  | 0.28  |  0.39  | 0.0081 
 75  | 0.12  |  0.18  | 0.0121 
100  | 0.08  |  0.14  | 0.0208 
125  | 0.04  |  0.08  | 0.0252 
150  | 0.03  |  0.08  | 0.0300 
175  | 0.02  |  0.06  | 0.0358 
200  | 0.02  |  0.05  | 0.0419 
225  | 0.01  |  0.04  | 0.0466 
250  | 0.01  |  0.03  | 0.0512 
275  | 0.01  |  0.04  | 0.0558 
300  | 0.00  |  0.03  | 0.0598 
325  | 0.00  |  0.03  | 0.0638 
350  | 0.00  |  0.02  | 0.0676 
375  | 0.00  |  0.02  | 0.0716 
400  | 0.00  |  0.01  | 0.0781 
425  | 0.00  |  0.02  | 0.0822 
450  | 0.00  |  0.01  | 0.0862 
  0.105009 seconds (146.70 k allocations: 24.021 MB, 4.10% gc time)
change rho density to n/s: 2
rademacher-50-5
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 0.00  |  0.01  | 0.0243 
 50  | 0.00  |  0.00  | 0.0458 
  0.049736 seconds (67.52 k allocations: 14.899 MB, 9.41% gc time)
rademacher-5

In [35]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [36]:
# Testing countmin 
options.sketchsize =10;#floor((prob.n)^(1/2));
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

countmin
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 12.63  |  7.22  | 0.0401 
 50  | 11.20  |  5.90  | 0.0834 
 75  | 10.43  |  5.93  | 0.1198 
100  | 9.92  |  5.30  | 0.1379 
125  | 9.60  |  5.16  | 0.1659 
150  | 9.30  |  5.07  | 0.1841 
175  | 9.08  |  4.94  | 0.2040 
200  | 8.87  |  4.80  | 0.2221 
225  | 8.94  |  4.76  | 0.2402 
250  | 8.53  |  4.89  | 0.2821 
275  | 8.39  |  5.06  | 0.3020 
300  | 8.26  |  4.94  | 0.3201 
325  | 8.15  |  5.23  | 0.3382 
350  | 8.01  |  5.23  | 0.3689 
375  | 7.92  |  5.15  | 0.3916 
400  | 7.81  |  4.95  | 0.4140 
425  | 7.73  |  4.69  | 0.4322 
450  | 7.15  |  4.16  | 0.4512 
475  | 7.59  |  4.88  | 0.5159 
500  | 7.42  |  4.52  | 0.5673 
525  | 7.46  |  4.53  | 0.5942 
550  | 7.41  |  4.47  | 0.6123 
575  | 7.36  |  4.46  | 0.6318 
600  | 7.30  |  4.41  | 0.6500 
625  | 7.24  |  4.40  | 0.6690 
650  | 7.19  |  4.39  | 0.7389 
675  | 7.15  |  4.44  | 0.7819 
700  | 7.10  |  4.42  | 0.8266 
725  | 7.06  |  

In [37]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 0.03  |  0.06  | 0.0226 
 50  | 0.00  |  0.02  | 0.0292 
 75  | 0.00  |  0.01  | 0.0297 
  0.040265 seconds (3.04 k allocations: 724.578 KB)


In [38]:
# Testing CD 
options.maxiter=5000;
options.skip_error_calculation = 250;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CD";# gradient descent
options.sketchsize = 50;
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-10
-------------------
It   | Error% | Residual |  Time   
-------------------
250  | 0.09  |  0.10  | 0.0462 
500  | 0.00  |  0.04  | 0.0950 
750  | 0.00  |  0.02  | 0.1434 
1000  | 0.00  |  0.01  | 0.2084 
  0.239952 seconds (36.45 k allocations: 14.408 MB, 1.82% gc time)
CD-50
-------------------
It   | Error% | Residual |  Time   
-------------------
250  | 0.00  |  0.00  | 0.0611 
  0.065304 seconds (10.11 k allocations: 22.947 MB, 5.74% gc time)


In [39]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [40]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

a9a


In [344]:
# Loading saved problem data
probname = "gisette_scale"#"gisette_scale";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];

In [125]:
OUTPUTS = []
options.tol = 10.0^(-2.0);
options.restol = 10.0^(-2.0);
options.maxiter=9000000;
options.max_time = 300;
options.skip_error_calculation = 100;
options.AUX = [0 10] # don't use sign flipping and use a density of 5 per row
method_name = "rademacher";# ROS
options.sketchsize = 100; #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.skip_error_calculation = 10;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.maxiter=5000;
options.skip_error_calculation = 500;
method_name = "CD";# gradient descent
options.sketchsize = 100;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
plot_outputs_Plots(OUTPUTS,savename)

rademacher-20-4
-------------------
It   | Error% | Residual |  Time   
-------------------
100  | 53.59  |  32.15  | 9.8752 
200  | 40.72  |  18.19  | 19.2877 
300  | 32.43  |  13.76  | 29.8606 
400  | 28.10  |  11.28  | 40.1743 
500  | 24.14  |  9.74  | 50.1443 
600  | 21.41  |  9.03  | 59.9515 
700  | 18.70  |  7.48  | 70.1964 
800  | 16.30  |  6.57  | 80.9024 
900  | 14.79  |  5.97  | 90.2410 
1000  | 13.84  |  5.52  | 100.5206 
1100  | 12.65  |  5.12  | 111.2109 
1200  | 11.32  |  4.75  | 124.0030 
1300  | 10.26  |  4.41  | 134.7009 
1400  | 9.48  |  4.12  | 144.6959 
1500  | 8.78  |  3.86  | 154.1839 
1600  | 8.19  |  3.61  | 163.3525 
1700  | 7.66  |  3.52  | 172.9010 
1800  | 7.64  |  3.28  | 183.0576 
1900  | 6.19  |  3.11  | 194.0066 
2000  | 6.48  |  2.90  | 203.9059 
2100  | 5.89  |  2.71  | 214.1605 
2200  | 5.55  |  2.52  | 224.1497 
2300  | 5.23  |  2.51  | 233.8348 
2400  | 4.98  |  2.34  | 243.3699 
2500  | 4.72  |  2.25  | 253.3765 
2600  | 4.49  |  2.14  | 263.1168 


In [133]:
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.multiclass


In [ ]:
options.maxiter=20000;
options.skip_error_calculation = 500;
method_name = "CD";# gradient descent
options.sketchsize = 100;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];


CD-100
-------------------
It   | Error% | Residual |  Time   
-------------------
500  | 9.33  |  5.29  | 45.1216 
